In [1]:
import pulp
import numpy as np

## MAKE-TO-ORDER PRODUCTION

Dataset (can be read from a spreadsheet):

In [2]:
l = np.array([[8, 8],
              [8, 8],
              [8, 8]]) # length of each shift s on day t (hours)

b = np.array([[[8, 8],
               [8, 8],
               [8, 8]],
              [[8, 8],
               [8, 8],
               [8, 8]],
              [[8, 8],
               [8, 8],
               [8, 8]]]) # available capacity of resource r in shift s on day t (hours)

# each order j follows a processing route where the operations have a linear precedence relationship
p = np.array([[[0, 0, 14],
               [12, 0, 0],
               [0, 14, 0]],
              [[0, 0, 14],
               [10, 0, 0],
               [0, 14, 0]],
              [[8, 0, 0],
               [0, 0, 6],
               [0, 8, 0]]]) # processing time of operation o on order j with resource r (hours)

d = np.array([3, 3, 3]) # due date of order j (days)

q = np.array([10000, 9800, 6500]) # selling price of order j (USD)

c = np.array([[200, 300],
              [200, 250],
              [200, 200]]) # cost of using resource r in shift s on day t (USD/hour)

In [3]:
J = list(range(p.shape[0])) # orders
O = list(range(p.shape[1])) # operations
R = list(range(p.shape[2])) # resources
T = list(range(l.shape[0])) # days
S = list(range(l.shape[1])) # shifts

Variables:

In [4]:
# hours operation o of order j is processed on resource r in shift s on day t
X = np.array([[[[[pulp.LpVariable('X' + ''.join([str(index)
                  for index in (j, o, r, t, s)]), lowBound=0, cat="Continuous")
                  for s in S] for t in T] for r in R] for o in O] for j in J])

# 1 if operation o of order j is processed on resource r in shift s on day t
# 0 otherwise
Y = np.array([[[[[pulp.LpVariable('Y' + ''.join([str(index)
                  for index in (j, o, r, t, s)]), cat="Binary")
                  for s in S] for t in T] for r in R] for o in O] for j in J])

# 1 if order j is selected
# 0 otherwise
U = np.array([pulp.LpVariable(str(j), cat="Binary") for j in J])

Defining the problem:

In [5]:
problem = pulp.LpProblem("make-to-order production", pulp.LpMaximize)

/opt/conda/lib/python3.7/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


First, we add the objective function to the problem:

In [6]:
problem += pulp.lpSum([q[j] * U[j] for j in J])\
         - pulp.lpSum([c[r, s] * X[j, o, r, t, s]
           for s in S for t in T for r in R for o in O for j in J])

Then we add all the constraints (we have already added the nonnegativity and integer constraints earlier, when we created the variables):

In [7]:
# whenever an operation is processed on a resource it should be processed \
# for at least "tau" hours (or other unit of time)
tau = 1 

constraints = []

constraints += [pulp.lpSum([X[j,o,r,t,s] for j in J for o in O]) <= b[r,t,s]
                for s in S for t in T for r in R] # (1)

constraints += [pulp.lpSum([X[j,o,r,t,s] for t in T for s in S]) >= p[j,o,r] * U[j]
                for r in R for o in O for j in J] # (2)

constraints += [pulp.lpSum([X[j,o,r,t,s] for o in O for r in R]) <= l[t,s]
                for s in S for t in T for j in J] # (3)

constraints += [X[j,o,r,t,s] >= tau*Y[j,o,r,t,s]
                for s in S for t in T for r in R for o in O for j in J] # (4)

constraints += [X[j,o,r,t,s] <= p[j,o,r] * Y[j,o,r,t,s]
                for s in S for t in T for r in R for o in O for j in J] # (5)

constraints += [pulp.lpSum([t*Y[j,len(O)-1,r,t,s] for r in R]) <= d[j] * U[j]
                for s in S for t in T for j in J] # (6)

constraints += [pulp.lpSum([X[j,o-1,r,t_,s_] for t_ in range(0, t-1) for s_ in S]) \
                + pulp.lpSum([X[j,o-1,r,t,s_] for s_ in range(0, s)]) >= \
                p[j,o-1,r] * pulp.lpSum([Y[j,o,r_,t,s] for r_ in R])
                for s in S[:-1] for t in T for r in R for o in O[1:] for j in J] # (7)
                
constraints += [pulp.lpSum([pulp.lpSum([X[j,o-1,r,t_,s] for t_ in range(0, t)]) for s in S]) >= \
                p[j,o-1,r] * pulp.lpSum([Y[j,o,r_,t,len(S)-1] for r_ in R])
                for t in T for r in R for o in O[1:] for j in J] # (8)

for constraint in constraints:
  problem += constraint

So the problem is the following:

In [8]:
print(problem)

make-to-order_production:
MAXIMIZE
10000*0 + 9800*1 + 6500*2 + -200*X00000 + -300*X00001 + -200*X00010 + -300*X00011 + -200*X00020 + -300*X00021 + -200*X00100 + -250*X00101 + -200*X00110 + -250*X00111 + -200*X00120 + -250*X00121 + -200*X00200 + -200*X00201 + -200*X00210 + -200*X00211 + -200*X00220 + -200*X00221 + -200*X01000 + -300*X01001 + -200*X01010 + -300*X01011 + -200*X01020 + -300*X01021 + -200*X01100 + -250*X01101 + -200*X01110 + -250*X01111 + -200*X01120 + -250*X01121 + -200*X01200 + -200*X01201 + -200*X01210 + -200*X01211 + -200*X01220 + -200*X01221 + -200*X02000 + -300*X02001 + -200*X02010 + -300*X02011 + -200*X02020 + -300*X02021 + -200*X02100 + -250*X02101 + -200*X02110 + -250*X02111 + -200*X02120 + -250*X02121 + -200*X02200 + -200*X02201 + -200*X02210 + -200*X02211 + -200*X02220 + -200*X02221 + -200*X10000 + -300*X10001 + -200*X10010 + -300*X10011 + -200*X10020 + -300*X10021 + -200*X10100 + -250*X10101 + -200*X10110 + -250*X10111 + -200*X10120 + -250*X10121 + -200*X10200 +

Solving the problem:

In [9]:
problem.solve()
print("State:", pulp.LpStatus[problem.status])
for v in problem.variables():
  print(v.name, "=", v.varValue)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.7/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/bc4fba5812a44bd5b5e3a378d1b8665c-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/bc4fba5812a44bd5b5e3a378d1b8665c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 518 COLUMNS
At line 2355 RHS
At line 2869 BOUNDS
At line 3035 ENDATA
Problem MODEL has 513 rows, 327 columns and 1341 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1700 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 10 strengthened rows, 5 substitutions
Cgl0004I processed model has 84 rows, 55 columns (28 integer (28 of which binary)) and 206 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -1700
Cbc0038I Relaxing continuous gives -1700
Cbc0038I Before mini branch and bound, 28 intege

In [10]:
print("Maximal profit: %d USD" % pulp.value(problem.objective))

Maximal profit: 1700 USD
